In [0]:
# We will use the official tokenization script created by the Google team
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 3.4MB/s 


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from sklearn import model_selection
from sklearn import metrics

import tokenization

## Using the original BERT code

Helper Functions

In [0]:
def bert_encode(texts, tokenizer, max_len=160):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
def build_model(bert_layer, max_len=160):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Load and Preprocess¶

In [6]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 22 s, sys: 4.25 s, total: 26.2 s
Wall time: 1min 11s


In [7]:
df1 = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)
df2 = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines=True)
# re-order attibute columns in df2
df2 = df2[['article_link','headline','is_sarcastic']]
df = pd.concat([df1, df2], axis=0)
df = df.drop(['article_link'], axis=1)
print(len(df))
df.head()

55328


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [0]:
df.reset_index(inplace=True, drop=True)

In [9]:
df.sort_values("headline", inplace = True)
df.drop_duplicates(subset ="headline", 
                     keep = 'first', inplace = True) 
print(len(df))

28503


In [10]:
print(df.headline.str.len().max()) # maximum headline has 926 characters

index_array = df.headline.str.len() <= 160 # cutting it off at tweet length

df = df[index_array]

train,test = model_selection.train_test_split(df,test_size = 0.3)

926


In [0]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
train_input = bert_encode(train.headline.values, tokenizer, max_len = 160)
test_input = bert_encode(test.headline.values, tokenizer, max_len = 160)
train_labels = train.is_sarcastic.values
test_labels = test.is_sarcastic.values

### Model: Build, Train, Predict, Validate¶

In [13]:
model = build_model(bert_layer, max_len = 160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [14]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.1,
    epochs=5,
    batch_size=20
)

model.save('model.h5')

Epoch 1/5
898/898 [==============================] - 1036s 1s/step - loss: 0.3412 - accuracy: 0.8483 - val_loss: 0.2834 - val_accuracy: 0.8787
Epoch 2/5
898/898 [==============================] - 1034s 1s/step - loss: 0.1698 - accuracy: 0.9339 - val_loss: 0.2438 - val_accuracy: 0.9078
Epoch 3/5
898/898 [==============================] - 1034s 1s/step - loss: 0.0746 - accuracy: 0.9768 - val_loss: 0.2941 - val_accuracy: 0.8942
Epoch 4/5
898/898 [==============================] - 1035s 1s/step - loss: 0.0224 - accuracy: 0.9955 - val_loss: 0.3354 - val_accuracy: 0.9098
Epoch 5/5
898/898 [==============================] - 1034s 1s/step - loss: 0.0050 - accuracy: 0.9991 - val_loss: 0.4189 - val_accuracy: 0.9078


In [0]:
test_pred = model.predict(test_input)
test_pred = test_pred.round().astype(int)

In [16]:
metrics.accuracy_score(test_labels,test_pred)

0.9114516317697976

In [18]:
metrics.f1_score(test_labels,test_pred)

0.9068307692307692

In [19]:
metrics.recall_score(test_labels,test_pred)

0.8956965718453683

In [20]:
metrics.precision_score(test_labels,test_pred)

0.9182452642073778

In [22]:
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
# kappa
kappa = cohen_kappa_score(test_labels,test_pred)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(test_labels,test_pred)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(test_labels,test_pred)
print(matrix)

Cohens kappa: 0.822491
ROC AUC: 0.910878
[[4108  328]
 [ 429 3684]]


# Extract FalsePositives and FalseNegatives


In [0]:
def getFP_FN_lists(test_X, test_y, pred_y):
    FP_text = []
    FP_index = []
    FN_text = []
    FN_index = []
    for i in range(len(test_y)):
        if(pred_y[i]==1 and test_y[test_y.index[i]]==0):
            FP_text.append(test['headline'][test_y.index[i]])
            FP_index.append(test_y.index[i])
        elif(pred_y[i]==0 and test_y[test_y.index[i]]==1):
            FN_text.append(test['headline'][test_y.index[i]])
            FN_index.append(test_y.index[i])
            
    return FP_text,FP_index,FN_text,FN_index

In [0]:
'''Returns 2 dataframes, one with all the False Positives and one with all the False Negatives'''
def getFP_FN(test_X, test_y, pred_y):
    FP_text,FP_index,FN_text,FN_index = getFP_FN_lists(test_X, test_y, pred_y)
    d_FP = {'FP_text':FP_text,'FP_index':FP_index}
    df_FP = pd.DataFrame(d_FP)
    d_FN = {'FN_text':FN_text,'FN_index':FN_index}
    df_FN = pd.DataFrame(d_FN)
    
    return df_FP,df_FN

In [0]:
# We get the FPs and FNs as DataFrames and store them to CSVs
df_FP,df_FN = getFP_FN(test['headline'], test['is_sarcastic'],test_pred)
df_FP.to_csv('bert_FP.csv', index=True)
df_FN.to_csv('bert_FN.csv', index=True)